In [1]:
import zlib
import gzip
import psycopg2
import pandas as pd

## 1. Connect with mimic database

In [2]:
conn = psycopg2.connect(host='madigun.asuscomm.com', port='5432', database='mimic', user='grace', password="grace0410")
cur = conn.cursor()

## 2. Fetch required data

In [3]:
# cur.execute('''SELECT
#                       e.subject_id,
#                       e.hadm_id,
#                       marital_status,
#                       ethnicity,
#                       gender,
#                       ROUND((cast(admittime AS DATE) - cast(dob AS DATE)) / 365, 2) AS admit_age,
#                       icd9_code
#                     FROM (SELECT
#                             subject_id,
#                             hadm_id,
#                             marital_status,
#                             ethnicity,
#                             admittime
#                           FROM admissions a
#                           WHERE admission_type IN ('EMERGENCY', 'URGENT')) e
#                       LEFT JOIN patients p
#                         ON e.subject_id = p.subject_id
#                       LEFT JOIN (SELECT *
#                                  FROM diagnoses_icd
#                                  WHERE seq_num = 1
#                                        AND icd9_code IS NOT NULL) AS d
#                         ON e.hadm_id = d.hadm_id;''')
# result = cur.fetchall()

In [4]:
df = pd.read_sql('''SELECT
                          e.subject_id,
                          e.hadm_id,
                          marital_status,
                          ethnicity,
                          gender,
                          ROUND((cast(admittime AS DATE) - cast(dob AS DATE)) / 365, 2) AS admit_age,
                          icd9_code
                        FROM (SELECT
                                subject_id,
                                hadm_id,
                                marital_status,
                                ethnicity,
                                admittime
                              FROM admissions a
                              WHERE admission_type IN ('EMERGENCY', 'URGENT')) e
                          LEFT JOIN patients p
                            ON e.subject_id = p.subject_id
                          LEFT JOIN (SELECT *
                                     FROM diagnoses_icd
                                     WHERE seq_num = 1
                                           AND icd9_code IS NOT NULL) AS d
                            ON e.hadm_id = d.hadm_id;''', conn)

In [5]:
df.head()

,subject_id,hadm_id,marital_status,ethnicity,gender,admit_age,icd9_code
0,2148,123870,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,M,59.0,49121
1,2149,128303,MARRIED,UNKNOWN/NOT SPECIFIED,M,54.0,431
2,2150,189980,MARRIED,WHITE,F,71.0,2230
3,2151,153468,SINGLE,WHITE,M,45.0,3570
4,2152,127190,MARRIED,UNKNOWN/NOT SPECIFIED,M,67.0,41401


In [9]:
#  LEFT JOIN 기준이기 때문에 icd9_code가 없어도 포함될 수 있다. 따라서 추가적으로 제거한다.
df[df.icd9_code.isnull()].shape

(0, 7)

In [7]:
df = df[~df.icd9_code.isnull()]
df[df.icd9_code.isnull()].shape

In [10]:
df.icd9_code.values

array(['49121', '431', '2230', ..., '85222', '5070', '86121'],
      dtype=object)

## 3. Get embeddings

In [55]:
DIMENSION = 300

with gzip.open('./data/claims_codes_hs_300.txt.gz', 'rb') as f:
    file_content = f.read()

In [56]:
file_content = file_content.split(b'\n')
file_content = file_content[2:]

In [57]:
file_content  = list(map(lambda x: x.split(b' '), file_content))
file_content = { i[0].decode('ASCII') : i[1:DIMENSION + 1] for i in file_content}

In [64]:
for ind, row in df[:2].iterrows():
    code = row['icd9_code']
    print(code)
    print(file_content['C_'+code])

49121


KeyError: 'C_49121'

In [69]:
len(df.icd9_code.unique())

2532

In [66]:
for grp, rows in df[:100].groupby('icd9_code'):
    print(grp)
    print(rows)

0051
    subject_id  hadm_id marital_status ethnicity gender  admit_age icd9_code
96        2081   109103       DIVORCED     OTHER      M       75.0      0051
0380
    subject_id  hadm_id marital_status ethnicity gender  admit_age icd9_code
85        2067   104215         SINGLE     WHITE      F       51.0      0380
03811
    subject_id  hadm_id marital_status ethnicity gender  admit_age icd9_code
77        2053   184822        MARRIED     WHITE      M       84.0     03811
03849
   subject_id  hadm_id marital_status ethnicity gender  admit_age icd9_code
8        2156   157872       DIVORCED     WHITE      M       59.0     03849
0388
    subject_id  hadm_id marital_status              ethnicity gender  \
29        2044   168888       DIVORCED  UNKNOWN/NOT SPECIFIED      F   

    admit_age icd9_code  
29       80.0      0388  
0389
    subject_id  hadm_id marital_status              ethnicity gender  \
12        2165   140032        MARRIED                  WHITE      M   
28        218